# Neural Network with Sklearn for Sentiment Analysis ( MLPClassifier )

In [1]:
# import library yang dibutuhkan

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import string
import re
from collections import Counter
from tqdm import tqdm

# Import sklearn modul

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
# from sklearn.externals import joblib

from gensim.models import Word2Vec
import pickle

#import sastrawi
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#tokenize
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Preprosses Data

stopwords_indonesia = list(set(stopwords.words('indonesian')))

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Tambahan stopword
# add_stopword = ['ya', 'sembilan','belas', 'haha', 'ribu', 'puluh', 'dua', 'sih', 'nya' ]

# stopwords_indonesia.extend(add_stopword)

def text_proses(teks):
  teks = teks.lower()
  teks = re.sub("@[A-Za-z0-9_]+"," ", teks) #Menghilangkan mention
  teks = re.sub("#[A-Za-z0-9_]+"," ", teks) #Menghilangkan hashtag
  teks = re.sub(r"\\n"," ",teks) #Menghilangkan \n
  teks = teks.strip()
  teks = re.sub(r"http\S+", " ", teks) #Menghilangkan link
  teks = re.sub(r"www.\S+", " ", teks) #Menghilangkan link
  teks = re.sub("[^A-Za-z\s']"," ", teks) #Menghilangkan yang bukan huruf
  teks = re.sub("rt"," ",teks) #Menghilangkan RT
  tokens = word_tokenize(teks)
  # teks = ' '.join([word for word in tokens if word not in stopwords_indonesia])
  teks = ' '.join([word for word in tokens ])
  teks = stemmer.stem(teks)
  return teks

In [2]:
# Upload Clean_data file

df = pd.read_csv('data_clean.csv')
df.head(1)

,text,label,text_clean,processed_text,pro_text
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung milik usaha pabrik puluh kenal putih ba...,warung milik usaha pabrik puluh kenal putih ba...,warung ini milik oleh usaha pabrik tahu yang s...


In [43]:
# df['text_clean'] = df.text.apply(lambda x:text_proses(x))

In [72]:
# df.to_csv('data_clean.csv', index=False)

In [71]:
# import time
# start_time = time.time()
# df['pro_text'] = [text_proses(x) for x in df['text']]
# end_time = time.time()
# print('lama proses : ', end_time - start_time)

lama proses :  1160.0575714111328


In [82]:
# df.loc[df.label == 'neutral']

In [83]:
# df.loc[df.pro_text.duplicated()]

In [5]:
df.pro_text.duplicated().sum(), df.pro_text.isna().sum()  # Data sudah clean

(23, 0)

## membuat vector untuk inputan ke Machine Learning

### 1. Dengan menggunakan Bag of Word ( CountVectorizer )

In [56]:
# Proses Feature extraction Bag of Word ( CountVectorizer )

data_preprocessed = df.pro_text.to_list()

count_vect = CountVectorizer(ngram_range=(1,2))
count_vect.fit(data_preprocessed)

X_bow = count_vect.transform(data_preprocessed)
print('Feature extraction selesai')

Feature extraction selesai


### Best Model for Bag of words

In [58]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.84      0.84      0.84       664
     neutral       0.79      0.76      0.77       200
    positive       0.92      0.93      0.92      1321

    accuracy                           0.89      2185
   macro avg       0.85      0.84      0.85      2185
weighted avg       0.89      0.89      0.89      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.83      0.85      0.84       663
     neutral       0.86      0.71      0.78       243
    positive       0.91      0.94      0.92      1279

    accuracy                           0.88      2185
   macro avg       0.87      0.83      0.85      2185
weighted avg       0.88      0.88      0.88      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.84      0.85      0.85       672
     neutral       0.87      0.75      0.81       237
    positive       0.92      0

In [60]:
with open('model_bow_88_pct.pickle', 'wb') as f:
    pickle.dump(count_vect, f)

In [92]:
# real world test
original_text = '''


         @DivHumas_Polri Diem loe,anak kecil'""|"

                '''

text = count_vect.transform([text_proses(original_text)])
result = clf.predict(text)[0]

print('Sentiment :\n')
print(result)

Sentiment :

negative


In [86]:
# Cross Validation dengan df.pro_text

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[5,10]

for j in tqdm(hd):
  for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))


    accuracies.append(accuracy)




  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

 50%|█████     | 1/2 [01:29<01:29, 89.82s/it]


jmlh hidden layer :  5
rata-rata akurasi :  0.882745995423341


100%|██████████| 2/2 [02:43<00:00, 81.89s/it]


jmlh hidden layer :  10
rata-rata akurasi :  0.8769336384439359


In [39]:
# Cross Validation dengan df.text

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[5,10]

for j in tqdm(hd):
  for iteration, data in enumerate(kf.split(X_bow), start=1):

    data_train = X_bow[data[0]]
    target_train = y[data[0]]

    data_test = X_bow[data[1]]
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))


    accuracies.append(accuracy)




  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

 50%|█████     | 1/2 [01:40<01:40, 100.78s/it]


jmlh hidden layer :  5
rata-rata akurasi :  0.8785354691075515


100%|██████████| 2/2 [03:09<00:00, 94.66s/it]


jmlh hidden layer :  10
rata-rata akurasi :  0.87720823798627


### 2. Dengan menggunakan Tf-IDF ( TfidfVectorizer )

In [68]:
data_preprocessed = df.pro_text.to_list()

In [69]:
# Proses Feature extraction Tf-IDF ( TfidfVectorizer )
tfidf = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(data_preprocessed)
print('Feature Extraction Selesai')

Feature Extraction Selesai


In [70]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in tqdm(enumerate(kf.split(np.array(data_preprocessed)), start=1)):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

1it [07:39, 459.02s/it]

Training ke- 1
              precision    recall  f1-score   support

    negative       0.83      0.81      0.82       664
     neutral       0.85      0.67      0.75       200
    positive       0.90      0.94      0.92      1321

    accuracy                           0.87      2185
   macro avg       0.86      0.80      0.83      2185
weighted avg       0.87      0.87      0.87      2185



2it [14:42, 438.30s/it]

Training ke- 2
              precision    recall  f1-score   support

    negative       0.83      0.82      0.82       663
     neutral       0.89      0.68      0.77       243
    positive       0.90      0.94      0.92      1279

    accuracy                           0.88      2185
   macro avg       0.87      0.81      0.84      2185
weighted avg       0.88      0.88      0.87      2185



3it [22:34, 453.58s/it]

Training ke- 3
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       672
     neutral       0.89      0.70      0.78       237
    positive       0.90      0.95      0.92      1276

    accuracy                           0.89      2185
   macro avg       0.88      0.83      0.85      2185
weighted avg       0.89      0.89      0.88      2185



4it [29:16, 433.33s/it]

Training ke- 4
              precision    recall  f1-score   support

    negative       0.83      0.84      0.83       713
     neutral       0.91      0.63      0.75       219
    positive       0.90      0.94      0.92      1253

    accuracy                           0.88      2185
   macro avg       0.88      0.80      0.83      2185
weighted avg       0.88      0.88      0.87      2185



5it [39:06, 469.33s/it]

Training ke- 5
              precision    recall  f1-score   support

    negative       0.82      0.85      0.83       695
     neutral       0.86      0.61      0.72       239
    positive       0.90      0.94      0.92      1251

    accuracy                           0.87      2185
   macro avg       0.86      0.80      0.82      2185
weighted avg       0.87      0.87      0.87      2185

rata-rata akurasi :  0.8766132723112129


In [71]:
with open('model_mlp_tfidf_87_pct.pickle', 'wb') as f:
    pickle.dump(tfidf, f)

In [93]:
# real world test
original_text = '''

    @DivHumas_Polri Diem loe,anak kecil'""|"


                '''

text = tfidf.transform([text_proses(original_text)])
result = clf.predict(text)[0]

print('Sentiment :\n')
print(result)

Sentiment :

negative


In [15]:
# Cross Validation df.text_clean

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[5,10]

for j in tqdm(hd):
  for iteration, data in (enumerate(kf.split(X_tfidf), start=1)):

    data_train = X_tfidf[data[0]]
    target_train = y[data[0]]

    data_test = X_tfidf[data[1]]
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

  0%|          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:32, 32.35s/it]
2it [00:48, 22.94s/it]
3it [01:10, 22.27s/it]
4it [01:28, 20.91s/it]
5it [01:39, 19.98s/it]
 50%|█████     | 1/2 [01:39<01:39, 99.92s/it]

jmlh hidden layer :  5
rata-rata akurasi :  0.8196796338672769



0it [00:00, ?it/s]
1it [00:17, 17.96s/it]
2it [00:37, 18.59s/it]
3it [00:54, 18.13s/it]
4it [01:12, 17.90s/it]
5it [01:33, 18.64s/it]
100%|██████████| 2/2 [03:13<00:00, 96.59s/it]

jmlh hidden layer :  10
rata-rata akurasi :  0.8225171624713961


In [33]:
# Cross Validation df.text

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[5,10]

for j in tqdm(hd):
  for iteration, data in (enumerate(kf.split(X_tfidf), start=1)):

    data_train = X_tfidf[data[0]]
    target_train = y[data[0]]

    data_test = X_tfidf[data[1]]
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

 50%|█████     | 1/2 [02:57<02:57, 177.13s/it]


jmlh hidden layer :  5
rata-rata akurasi :  0.8624256292906178


100%|██████████| 2/2 [06:05<00:00, 182.78s/it]


jmlh hidden layer :  10
rata-rata akurasi :  0.8538215102974828


In [89]:
# Cross Validation df.pro_text

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[5,10]

for j in tqdm(hd):
  for iteration, data in (enumerate(kf.split(data_preprocessed), start=1)):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

 50%|█████     | 1/2 [02:03<02:03, 123.84s/it]


jmlh hidden layer :  5
rata-rata akurasi :  0.8675514874141875


100%|██████████| 2/2 [04:50<00:00, 145.38s/it]


jmlh hidden layer :  10
rata-rata akurasi :  0.8704805491990847


### 3. Dengan menggunakan gensim model ( word2vec )

In [34]:
# Preprocess the text (tokenization)
nltk.download('punkt')
corpus = [nltk.word_tokenize(doc) for doc in data_preprocessed]

# Train Word2Vec model
model = Word2Vec(corpus, vector_size=400, window=5, min_count=1, workers=4)

# Generate document embeddings
X_w2v = [model.wv[doc].mean(axis=0) for doc in corpus]

print('Feature Extraction Selesai')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Feature Extraction Selesai


In [37]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[300]

for j in tqdm(hd):
  for iteration, data in (enumerate(kf.split(X_w2v), start=1)):

    data_train = np.array(X_w2v)[data[0]]
    target_train = np.array(y)[data[0]]

    data_test = np.array(X_w2v)[data[1]]
    target_test = np.array(y)[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

100%|██████████| 1/1 [00:46<00:00, 46.63s/it]


jmlh hidden layer :  300
rata-rata akurasi :  0.822883295194508


In [90]:
# Menggunakan data df.pro_text
data_preprocessed = df.pro_text.to_list()

# Preprocess the text (tokenization)
nltk.download('punkt')
corpus = [nltk.word_tokenize(doc) for doc in data_preprocessed]

# Train Word2Vec model
model = Word2Vec(corpus, vector_size=400, window=5, min_count=1, workers=4)

# Generate document embeddings
X_w2v = [model.wv[doc].mean(axis=0) for doc in corpus]

print('Feature Extraction Selesai')

# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label
hd=[50,100,200,300,500,750]

for j in tqdm(hd):
  for iteration, data in (enumerate(kf.split(X_w2v), start=1)):

    data_train = np.array(X_w2v)[data[0]]
    target_train = np.array(y)[data[0]]

    data_test = np.array(X_w2v)[data[1]]
    target_test = np.array(y)[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=hd ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    # print('Training ke-', iteration)
    # print(classification_report(target_test, preds))

    accuracies.append(accuracy)

  average_accuracy = np.mean(accuracies)
  print('\njmlh hidden layer : ', j)
  # print('\n\n\n')
  print('rata-rata akurasi : ', average_accuracy)
  print('='*100)

"""

Feature Extraction Selesai
 17%|█▋        | 1/6 [03:58<19:51, 238.37s/it]
jmlh hidden layer :  50
rata-rata akurasi :  0.8291075514874141
====================================================================================================
 33%|███▎      | 2/6 [08:40<17:35, 263.97s/it]
jmlh hidden layer :  100
rata-rata akurasi :  0.8314416475972539
====================================================================================================
 50%|█████     | 3/6 [13:28<13:45, 275.15s/it]
jmlh hidden layer :  200
rata-rata akurasi :  0.8311212814645308
====================================================================================================
 67%|██████▋   | 4/6 [17:40<08:52, 266.03s/it]
jmlh hidden layer :  300
rata-rata akurasi :  0.8313729977116704
====================================================================================================
 83%|████████▎ | 5/6 [22:05<04:25, 265.68s/it]
jmlh hidden layer :  500
rata-rata akurasi :  0.8303524027459953
====================================================================================================
100%|██████████| 6/6 [26:29<00:00, 264.92s/it]
jmlh hidden layer :  750
rata-rata akurasi :  0.8305110602593438
====================================================================================================

"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Feature Extraction Selesai


 17%|█▋        | 1/6 [03:58<19:51, 238.37s/it]


jmlh hidden layer :  50
rata-rata akurasi :  0.8291075514874141


 33%|███▎      | 2/6 [08:40<17:35, 263.97s/it]

...


jmlh hidden layer :  300
rata-rata akurasi :  0.8313729977116704


 83%|████████▎ | 5/6 [22:05<04:25, 265.68s/it]


jmlh hidden layer :  500
rata-rata akurasi :  0.8303524027459953


100%|██████████| 6/6 [26:29<00:00, 264.92s/it]


jmlh hidden layer :  750
rata-rata akurasi :  0.8305110602593438


In [113]:
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print(classification_report(target_test, preds))

              precision    recall  f1-score   support

    negative       0.80      0.67      0.73       695
     neutral       0.64      0.74      0.69       239
    positive       0.86      0.92      0.89      1251

    accuracy                           0.82      2185
   macro avg       0.77      0.78      0.77      2185
weighted avg       0.82      0.82      0.82      2185



In [109]:
# test dengan real world case

original_text = '''

hoax hoax


                '''
# nltk.download('punkt')
text = [nltk.word_tokenize(original_text)]
model_w2v = Word2Vec(text, vector_size=400, window=5, min_count=1, workers=4)
data_w2v = [model_w2v.wv[doc].mean(axis=0) for doc in text]
result = clf.predict(data_w2v)[0]

print('Sentiment :\n')
print(result)

Sentiment :

positive


## 4. Kombinasi Bag of word & Tfidf

In [68]:
df = pd.read_csv('data_clean.csv')
df.head()

,text,label,text_clean,processed_text,pro_text
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung milik usaha pabrik puluh kenal putih ba...,warung milik usaha pabrik puluh kenal putih ba...,warung ini milik oleh usaha pabrik tahu yang s...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus k mmbri hujjah pa ai diwlh s...,mohon ulama lurus k mmbri hujjah pa ai diwlh s...,mohon ulama lurus dan k mmbri hujjah pa ai apa...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis jalan sumatera bandung nya ny...,lokasi strategis jalan sumatera bandung nya ny...,lokasi strategis di jalan sumatera bandung tem...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya unboxing paket barang nya b...,betapa bahagia nya unboxing paket barang nya b...,betapa bahagia nya diri ini saat unboxing pake...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh mahasiswa sombong kasih ka u kuning ajar u...,duh mahasiswa sombong kasih ka u kuning ajar u...,duh jadi mahasiswa jangan sombong dong kasih k...


In [69]:
X_train, X_test, y_train, y_test = train_test_split(df.pro_text, df.label, test_size=0.25, random_state=46, stratify=df.label)



In [70]:
Vectorize = CountVectorizer(ngram_range=(1,1))
X_train_vec = Vectorize.fit_transform(X_train)
X_test_vec = Vectorize.transform(X_test)

X_train_vec

<8193x10997 sparse matrix of type '<class 'numpy.int64'>'
	with 197209 stored elements in Compressed Sparse Row format>

In [51]:
with open ('Vectorize.pickle','wb') as f:
  pickle.dump(Vectorize,f)

In [53]:
with open ('Vectorize.pickle', 'rb') as f:
  vectorx = pickle.load(f)

In [55]:
# vectorx.vocabulary_.keys()

In [23]:
# print(X_train_vec)

In [22]:
# Vectorize.vocabulary_.keys()

In [71]:
Transformer = TfidfTransformer()

X_train_tfidf = Transformer.fit_transform(X_train_vec)
X_test_tfidf = Transformer.transform(X_test_vec)

X_train_tfidf

<8193x10997 sparse matrix of type '<class 'numpy.float64'>'
	with 197209 stored elements in Compressed Sparse Row format>

In [67]:
with open ('Transformer.pickle','wb') as f:
  pickle.dump(Transformer,f)

In [28]:
# print(X_train_tfidf)

In [72]:
# use ngram_range=(1,2)
import time

start_ = time.time()
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),early_stopping=True)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred))

Waktu training - predict : 21.294421195983887
              precision    recall  f1-score   support

    negative       0.77      0.89      0.83       852
     neutral       0.84      0.69      0.76       285
    positive       0.93      0.89      0.91      1595

    accuracy                           0.87      2732
   macro avg       0.85      0.82      0.83      2732
weighted avg       0.87      0.87      0.87      2732



In [78]:
# dir(clf)
# clf.__sizeof__()

In [79]:
with open('modelMLP.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [40]:
# use ngram_range=(1,1)
import time

start_ = time.time()
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),early_stopping=True)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred))

Waktu training - predict : 24.302531003952026
              precision    recall  f1-score   support

    negative       0.85      0.82      0.83       852
     neutral       0.81      0.75      0.78       285
    positive       0.91      0.93      0.92      1595

    accuracy                           0.88      2732
   macro avg       0.85      0.84      0.84      2732
weighted avg       0.88      0.88      0.88      2732



In [41]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train_vec = Vectorize.fit_transform(data_train)
    data_train_tfidf = Transformer.fit_transform(data_train_vec)
    target_train = y[data[0]]



    data_test = np.array(data_preprocessed)[data[1]]
    data_test_vec = Vectorize.transform(data_test)
    data_test_tfidf = Transformer.transform(data_test_vec)
    target_test = y[data[1]]



    clf = MLPClassifier(hidden_layer_sizes=(128,128,128),random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train_tfidf, target_train)
    preds = clf.predict(data_test_tfidf)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.80      0.82      0.81       664
     neutral       0.87      0.58      0.69       200
    positive       0.89      0.92      0.91      1321

    accuracy                           0.86      2185
   macro avg       0.85      0.77      0.80      2185
weighted avg       0.86      0.86      0.86      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.81      0.82      0.82       663
     neutral       0.86      0.70      0.78       243
    positive       0.90      0.92      0.91      1279

    accuracy                           0.87      2185
   macro avg       0.86      0.82      0.83      2185
weighted avg       0.87      0.87      0.87      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.80      0.86      0.83       672
     neutral       0.86      0.68      0.76       237
    positive       0.91      0

In [42]:
with open('model_mlp_kombinasi_87_pct.pickle', 'wb') as f:
    pickle.dump(clf, f)



NameError: ignored

In [47]:
# real world test
original_text = '''

|""b'@Ibboyjr_92 @kotaserui @MCAOps @BareskrimPolri @DivHumas_Polri @jokowi @Info_BinaMarga Barelang di Batam, alhamdull\xe2\x80\xa6 https://t.co/NGGZwfEbXZ'""|"



                '''

text = Vectorize.transform([text_proses(original_text)])
text = Transformer.transform(text)
result = clf.predict(text)[0]

print('Sentiment :\n')
print(result)

Sentiment :

neutral


### Melakukan proses pengecekan vector

In [ ]:
# Melakukan pengecekan panjang vector masing2
print('Panjang vector X_bow : ', len(X_bow.toarray()) )
print('Panjang vector X_tfidf : ', len(X_tfidf.toarray()) )
print('Panjang vector X_w2v : ', len(np.array(X_w2v)) )


  0%|          | 0/2 [00:00<?, ?it/s]